In [1]:
import re
from collections import Counter
import numpy as np
import pandas as pd

## load shakespeare sentences

In [11]:
def process_data(file):
 
    with open(file) as f:
        data = f.read()
    data_lower = data.lower()
    words = re.findall("\w+",data_lower)
    return words


In [12]:
word_l = process_data('../data/shakespeare.txt')
vocab = set(word_l) 
print(f"The first ten words in the text are: \n{word_l[0:10]}")
print(f"There are {len(vocab)} unique words in the vocabulary.")

The first ten words in the text are: 
['o', 'for', 'a', 'muse', 'of', 'fire', 'that', 'would', 'ascend', 'the']
There are 6116 unique words in the vocabulary.


## find probability of randomly select words 

In [13]:
def get_count(word_l):
    word_count_dict = Counter(word_l)
    
    return word_count_dict

In [15]:
word_count_dict = get_count(word_l)
print(f"There are {len(word_count_dict)} key values pairs")
print(f"The count for the word 'fire' is {word_count_dict.get('fire',0)}")

There are 6116 key values pairs
The count for the word 'fire' is 22


In [26]:
def get_probs(word_count_dict):
    probs = {}
    M = sum(word_count_dict.values())
    for key in word_count_dict.keys():
        #find probability of randomly select this word in corpus
        probs[key] = word_count_dict[key] / M
    return probs

In [29]:
probs = get_probs(word_count_dict)
print(f"Length of probs is {len(probs)}")
print(f"P('fire') is {probs['fire']:.6f}")

Length of probs is 6116
P('fire') is 0.000410


## string manipulation
* delete_letter: given a word, it returns all the possible strings that have one character removed.
* switch_letter: given a word, it returns all the possible strings that have two adjacent letters switched.
* replace_letter: given a word, it returns all the possible strings that have one character replaced by another different letter.
* insert_letter: given a word, it returns all the possible strings that have an additional character inserted.

### delete_letter operation

In [34]:
def delete_letter(word, verbose=False):
    
    delete_l = []
    delete_l = [word[:i-1]+word[i:] for i in range(len(word)+1) if i]
    if verbose: print(f"input word {word}, \ndelete_l = {delete_l}")

    return delete_l

In [35]:
delete_word_l = delete_letter(word="cans",verbose=True)

input word cans, 
delete_l = ['ans', 'cns', 'cas', 'can']


### switch_letter operation

In [56]:
def switch_letter(word, verbose=False):
    switch_l  = [word[:i]+word[i+1]+word[i]+word[i+2:] for i in range(len(word)+1) if(i<len(word)-1)]
    if verbose: print(f"Input word = {word} , \nswitch_l = {switch_l}") 

    return switch_l

In [57]:
switch_word_l = switch_letter(word="eta",verbose=True)

Input word = eta , 
switch_l = ['tea', 'eat']


### replace_letter operation

In [62]:
def replace_letter(word, verbose=False):

    letters = 'abcdefghijklmnopqrstuvwxyz'
    replace_l = [word[:i]+letter+word[i+1:]  for i in range(len(word)) for letter in letters]
    replace_l.remove(word)
    if verbose: print(f"Input word = {word}, \nreplace_l {replace_l}")   
    
    return replace_l

In [63]:
replace_l = replace_letter(word='can',verbose=True)

Input word = can, 
replace_l ['aan', 'ban', 'dan', 'ean', 'fan', 'gan', 'han', 'ian', 'jan', 'kan', 'lan', 'man', 'nan', 'oan', 'pan', 'qan', 'ran', 'san', 'tan', 'uan', 'van', 'wan', 'xan', 'yan', 'zan', 'can', 'cbn', 'ccn', 'cdn', 'cen', 'cfn', 'cgn', 'chn', 'cin', 'cjn', 'ckn', 'cln', 'cmn', 'cnn', 'con', 'cpn', 'cqn', 'crn', 'csn', 'ctn', 'cun', 'cvn', 'cwn', 'cxn', 'cyn', 'czn', 'caa', 'cab', 'cac', 'cad', 'cae', 'caf', 'cag', 'cah', 'cai', 'caj', 'cak', 'cal', 'cam', 'can', 'cao', 'cap', 'caq', 'car', 'cas', 'cat', 'cau', 'cav', 'caw', 'cax', 'cay', 'caz']


### insert letter operation

In [66]:
def insert_letter(word, verbose=False):

    letters = 'abcdefghijklmnopqrstuvwxyz'
    insert_l= replace_l = [word[:i]+letter+word[i:]  for i in range(len(word)+1) for letter in letters]
    if verbose: print(f"Input word {word}, \ninsert_l = {insert_l}")
    
    return insert_l


In [67]:
insert_l = insert_letter('at', True)
print(f"Number of strings output by insert_letter('at') is {len(insert_l)}")

Input word at, 
insert_l = ['aat', 'bat', 'cat', 'dat', 'eat', 'fat', 'gat', 'hat', 'iat', 'jat', 'kat', 'lat', 'mat', 'nat', 'oat', 'pat', 'qat', 'rat', 'sat', 'tat', 'uat', 'vat', 'wat', 'xat', 'yat', 'zat', 'aat', 'abt', 'act', 'adt', 'aet', 'aft', 'agt', 'aht', 'ait', 'ajt', 'akt', 'alt', 'amt', 'ant', 'aot', 'apt', 'aqt', 'art', 'ast', 'att', 'aut', 'avt', 'awt', 'axt', 'ayt', 'azt', 'ata', 'atb', 'atc', 'atd', 'ate', 'atf', 'atg', 'ath', 'ati', 'atj', 'atk', 'atl', 'atm', 'atn', 'ato', 'atp', 'atq', 'atr', 'ats', 'att', 'atu', 'atv', 'atw', 'atx', 'aty', 'atz']
Number of strings output by insert_letter('at') is 78


## Edit distance 

### Edit_one_letter

In [68]:
def edit_one_letter(word, allow_switches = True):

    edit_one_set = set()

    edit_one_set.update(delete_letter(word))
    if allow_switches:
        edit_one_set.update(switch_letter(word))
    edit_one_set.update(replace_letter(word))
    edit_one_set.update(insert_letter(word))

    return edit_one_set

In [70]:
tmp_word = "at"
tmp_edit_one_set = edit_one_letter(tmp_word)
# turn this into a list to sort it, in order to view it
tmp_edit_one_l = sorted(list(tmp_edit_one_set))

print(f"input word {tmp_word} \nedit_one_l \n{tmp_edit_one_l}\n")
print(f"The type of the returned object should be a set {type(tmp_edit_one_set)}")
print(f"Number of outputs from edit_one_letter('at') is {len(edit_one_letter('at'))}")

input word at 
edit_one_l 
['a', 'aa', 'aat', 'ab', 'abt', 'ac', 'act', 'ad', 'adt', 'ae', 'aet', 'af', 'aft', 'ag', 'agt', 'ah', 'aht', 'ai', 'ait', 'aj', 'ajt', 'ak', 'akt', 'al', 'alt', 'am', 'amt', 'an', 'ant', 'ao', 'aot', 'ap', 'apt', 'aq', 'aqt', 'ar', 'art', 'as', 'ast', 'at', 'ata', 'atb', 'atc', 'atd', 'ate', 'atf', 'atg', 'ath', 'ati', 'atj', 'atk', 'atl', 'atm', 'atn', 'ato', 'atp', 'atq', 'atr', 'ats', 'att', 'atu', 'atv', 'atw', 'atx', 'aty', 'atz', 'au', 'aut', 'av', 'avt', 'aw', 'awt', 'ax', 'axt', 'ay', 'ayt', 'az', 'azt', 'bat', 'bt', 'cat', 'ct', 'dat', 'dt', 'eat', 'et', 'fat', 'ft', 'gat', 'gt', 'hat', 'ht', 'iat', 'it', 'jat', 'jt', 'kat', 'kt', 'lat', 'lt', 'mat', 'mt', 'nat', 'nt', 'oat', 'ot', 'pat', 'pt', 'qat', 'qt', 'rat', 'rt', 'sat', 'st', 't', 'ta', 'tat', 'tt', 'uat', 'ut', 'vat', 'vt', 'wat', 'wt', 'xat', 'xt', 'yat', 'yt', 'zat', 'zt']

The type of the returned object should be a set <class 'set'>
Number of outputs from edit_one_letter('at') is 130


### Edit_two_letter
Now we can generalize this to implement to get two edits on a word. To do so, you would have to get all the possible edits on a single word and then for each modified word, you would have to modify it again.

In [71]:
def edit_two_letters(word, allow_switches = True):
    edit_two_set = set()
   
    edit_one = edit_one_letter(word,allow_switches=allow_switches)
    for w in edit_one:
        if w:
            edit_two_set.update(edit_one_letter(w,allow_switches=allow_switches)) 

    return edit_two_set

In [72]:
tmp_edit_two_set = edit_two_letters("a")
tmp_edit_two_l = sorted(list(tmp_edit_two_set))
print(f"Number of strings with edit distance of two: {len(tmp_edit_two_l)}")
print(f"First 10 strings {tmp_edit_two_l[:10]}")
print(f"Last 10 strings {tmp_edit_two_l[-10:]}")
print(f"The data type of the returned object should be a set {type(tmp_edit_two_set)}")
print(f"Number of strings that are 2 edit distances from 'at' is {len(edit_two_letters('at'))}")

Number of strings with edit distance of two: 2654
First 10 strings ['', 'a', 'aa', 'aaa', 'aab', 'aac', 'aad', 'aae', 'aaf', 'aag']
Last 10 strings ['zv', 'zva', 'zw', 'zwa', 'zx', 'zxa', 'zy', 'zya', 'zz', 'zza']
The data type of the returned object should be a set <class 'set'>
Number of strings that are 2 edit distances from 'at' is 7154


## suggest spelling suggestions

In [96]:
def get_corrections(word, probs, vocab, n=2, verbose = False):

    suggestions = []
    n_best = []
    print(word)
    suggestions = list((word in vocab and word) or edit_one_letter(word, False).intersection(vocab) or edit_two_letters(word, False).intersection(vocab))
    suggestions = list(set(suggestions))
    if verbose: print("suggestions = ", suggestions)

    n_best = [[s,probs[s]] for s in list(reversed(suggestions))]

    return n_best

In [81]:
my_word = 'dys' 
tmp_corrections = get_corrections(my_word, probs, vocab, 2, verbose=False)
for i, word_prob in enumerate(tmp_corrections):
    print(f"word {i}: {word_prob[0]}, probability {word_prob[1]:.6f}")


word 0: dye, probability 0.000019
word 1: days, probability 0.000410


## Minimum Edit distance

### Dynamic Programming
I use dynamic programming to find similarity between word and my corrections.Dynamic Programming breaks a problem down into subproblems which can be combined to form the final solution

In [82]:
def min_edit_distance(source, target, ins_cost = 1, del_cost = 1, rep_cost = 2):
    '''
    Input: 
        source: a string corresponding to the string you are starting with
        target: a string corresponding to the string you want to end with
        ins_cost: an integer setting the insert cost
        del_cost: an integer setting the delete cost
        rep_cost: an integer setting the replace cost
    Output:
        D: a matrix of len(source)+1 by len(target)+1 containing minimum edit distances
        med: the minimum edit distance (med) required to convert the source string to the target
    '''
    m = len(source)
    n = len(target)
    D = np.zeros((m+1, n+1), dtype=int) 
    for row in range(1,m+1): 
        D[row,0] = D[row-1,0] + del_cost
        
    for col in range(1,n+1): 
        D[0,col] = D[0,col-1] + ins_cost
        
    for row in range(1,m+1): 
        for col in range(1,n+1):
            r_cost = rep_cost
            if source[row-1] == target[col-1]:
                r_cost = 0
            D[row,col] = min([D[row-1,col]+del_cost, D[row,col-1]+ins_cost, D[row-1,col-1]+r_cost])

    med = D[m,n]

    return D, med


In [83]:
source =  'play'
target = 'stay'
matrix, min_edits = min_edit_distance(source, target)
print("minimum edits: ",min_edits, "\n")
idx = list('#' + source)
cols = list('#' + target)
df = pd.DataFrame(matrix, index=idx, columns= cols)
print(df)

minimum edits:  4 

   #  s  t  a  y
#  0  1  2  3  4
p  1  2  3  4  5
l  2  3  4  5  6
a  3  4  5  4  5
y  4  5  6  5  4


In [116]:
my_word = 'mozz' 
tmp_corrections = get_corrections(my_word, probs, vocab, 2, verbose=True)
suggestion_list=[]
for i, word_prob in enumerate(tmp_corrections):
    _, min_edits = min_edit_distance(my_word, word_prob[0])
    print(f"word {i}: {word_prob[0]}, probability {word_prob[1]:.6f}, min_edits {min_edits}")
    suggestion_list.append((word_prob[0],word_prob[1],min_edits))

mozz
suggestions =  ['mort', 'move', 'moon', 'mov', 'mock', 'most', 'mood', 'ooze', 'morn', 'mow', 'more', 'moan']
word 0: moan, probability 0.000093, min_edits 4
word 1: more, probability 0.003022, min_edits 4
word 2: mow, probability 0.000019, min_edits 3
word 3: morn, probability 0.000056, min_edits 4
word 4: ooze, probability 0.000019, min_edits 4
word 5: mood, probability 0.000019, min_edits 4
word 6: most, probability 0.001567, min_edits 4
word 7: mock, probability 0.000168, min_edits 4
word 8: mov, probability 0.000037, min_edits 3
word 9: moon, probability 0.000056, min_edits 4
word 10: move, probability 0.000112, min_edits 4
word 11: mort, probability 0.000019, min_edits 4


### suggest word with minimum edit distance and highe probability

In [117]:
tmp = min(suggestion_list,key = lambda t: t[2])[2]

In [120]:
nominates = [item for item in suggestion_list if item[2] == tmp]

In [122]:
max(nominates,key = lambda t: t[1])

('mov', 3.730368933487522e-05, 3)